### PULP. Minimizar gastos en el proceso de recoleccion de leche de las granjas en cochabamba.

La empresa PIL para satisfacer el abastecimiento de leche cruda contrata servicios de terceros (choferes que tienen sus propios camiones). 

este proceso de recoleccion de leche de las grangas productoras se lo realiza de lunes a viernes. 

la empresa tiene una lista de los posibles choferes y los montos que estos cobran por dia que varian por factores como el tamaño del carro que estos poseen.

### Objetivo
minimizar los costos en el proceso de contratacion analizando las restricciones de disponibilidad de cada chofer para el dia de la semana y la tarifa de los mismos.

In [2]:
# instalamos la libreria PULP
!pip install pulp

In [3]:
# importamos librerias
import pulp as pl
import collections as cl

In [4]:
# requerimiento de choferes por dia lunes=1 martes=2 miercoles=2 jueves=3 viernes=5
# El requerimiento de choferes por dia de la semana varia debido la necesidad/demanda de leche de la empresa.
requerimiento_choferes = [1, 2, 2, 3, 5]

# listado de choferes y su disponibilidad segun dia de la semana
choferes = {
    "Vladimir": {
        "disponibilidad": [0, 1, 4], #0=lunes 1=martes 2=miercoles ....
        "tarifa": 200
    },
    "Jonathan": {
        "disponibilidad": [1, 2, 3, 4],
        "tarifa": 150
    },
    "Ariel": {
        "disponibilidad": [2, 3],
        "tarifa": 350
    },
    "Fabiana": {
        "disponibilidad": [3, 4],
        "tarifa": 350
    },
    "Roger": {
        "disponibilidad": [1, 3, 4],
        "tarifa": 100
    },
    "Gonzalo": {
        "disponibilidad": [0, 2, 4],
        "tarifa": 250
    },
    "Eduardo": {
        "disponibilidad": [1, 3, 4],
        "tarifa": 300
    },
    "Armando": {
        "disponibilidad": [1, 2, 4],
        "tarifa": 200
    },
    "Carlos": {
        "disponibilidad": [0, 1, 3],
        "tarifa": 200
    }
}

In [5]:
# definimos que queremos minimizar el costo
problema = pl.LpProblem("scheduling", pl.LpMinimize)

In [6]:
# algunos modelos de variable
costos = []
choferes_por_dia = cl.defaultdict(list)

for chofer, info in choferes.items():
    for dia in info['disponibilidad']:
        chofer_dia = pl.LpVariable("%s_%s" % (chofer, dia), 0, 1, pl.LpInteger)
        choferes_por_dia[dia].append(chofer_dia)
        costos.append(chofer_dia * info['tarifa'])


In [7]:
# agregamos el objetivo: costo total (suma de tarifas)
problema += sum(costos)

In [8]:
# indicamos los requerimientos por dia.
for dia, requerimiento in enumerate(requerimiento_choferes):
    problema += sum(choferes_por_dia[dia]) >= requerimiento

In [9]:
# resolvemos el problema
status = problema.solve()
print("Resultado:", pl.LpStatus[status])

Resultado: Optimal


In [10]:
resultados = []
for dia, vars in choferes_por_dia.items():
    resultados.append({
        "dia": dia,
        "choferes": [var.name for var in vars if var.varValue == 1], #varValue estara en 1 si el chofer trabajara ese dia.
    })

for resultado in sorted(resultados, key=lambda x: x['dia']):
    print("dia:", resultado['dia'], 'choferes:', ', '.join(resultado['choferes']))

dia: 0 choferes: Carlos_0
dia: 1 choferes: Jonathan_1, Roger_1
dia: 2 choferes: Jonathan_2, Armando_2
dia: 3 choferes: Jonathan_3, Roger_3, Carlos_3
dia: 4 choferes: Vladimir_4, Jonathan_4, Roger_4, Gonzalo_4, Armando_4
